In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-22 03:12:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.05MB/s    in 0.2s    

2020-11-22 03:12:15 (5.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://mkangbucket.s3.amazonaws.com/jeopardy_new(2).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("jeopardy_new(2).csv"), sep=",", header=True)

# Show DataFrame
df.show()
# df.select(" Value").distinct().show()




+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|show_number|           round|  air_date|            category|value|year|month|            question|              answer|
+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+
|       5147|Double Jeopardy!| 1/16/2007|                1957| 1600|2007|    1|"Jack Kerouac pub...| & Nevil Shute wr...|
|       5147|Double Jeopardy!| 1/16/2007|    U.S. STATE NAMES| 1600|2007|    1|"This state's nam...|               Maine|
|       5147|Double Jeopardy!| 1/16/2007|     BODIES OF WATER| 2000|2007|    1|It sounds servile...|    Great Slave Lake|
|       5147|Double Jeopardy!| 1/16/2007|  ENTERTAINING WOMEN| 2000|2007|    1|"Artie Shaw was m...|         Ava Gardner|
|       5147|Double Jeopardy!| 1/16/2007|        SPEED HAMLET| 1000|2007|    1|He's the Fresh Pr...|          Fortinbras|
|       5147|Double Jeop

In [7]:
# import types
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [8]:
# print schema
df.printSchema()

root
 |-- show_number: string (nullable = true)
 |-- round: string (nullable = true)
 |-- air_date: string (nullable = true)
 |-- category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)



In [9]:
 from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['category']))
data_df.show()

+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+------+
|show_number|           round|  air_date|            category|value|year|month|            question|              answer|length|
+-----------+----------------+----------+--------------------+-----+----+-----+--------------------+--------------------+------+
|       5147|Double Jeopardy!| 1/16/2007|                1957| 1600|2007|    1|"Jack Kerouac pub...| & Nevil Shute wr...|     4|
|       5147|Double Jeopardy!| 1/16/2007|    U.S. STATE NAMES| 1600|2007|    1|"This state's nam...|               Maine|    16|
|       5147|Double Jeopardy!| 1/16/2007|     BODIES OF WATER| 2000|2007|    1|It sounds servile...|    Great Slave Lake|    15|
|       5147|Double Jeopardy!| 1/16/2007|  ENTERTAINING WOMEN| 2000|2007|    1|"Artie Shaw was m...|         Ava Gardner|    18|
|       5147|Double Jeopardy!| 1/16/2007|        SPEED HAMLET| 1000|2007|    1|He's the Fresh Pr.

In [11]:
hash_df = data_df[["round", "category", "length"]]
hash_df.show()

+----------------+--------------------+------+
|           round|            category|length|
+----------------+--------------------+------+
|Double Jeopardy!|                1957|     4|
|Double Jeopardy!|    U.S. STATE NAMES|    16|
|Double Jeopardy!|     BODIES OF WATER|    15|
|Double Jeopardy!|  ENTERTAINING WOMEN|    18|
|Double Jeopardy!|        SPEED HAMLET|    12|
|Double Jeopardy!|THAT'S SOME OF YO...|    28|
|Double Jeopardy!|                1957|     4|
|Double Jeopardy!|    U.S. STATE NAMES|    16|
|       Jeopardy!|          ADDRESS ME|    10|
|       Jeopardy!|  FILL IN THE ANIMAL|    18|
|       Jeopardy!|THE EMPIRE STRIKE...|    25|
|       Jeopardy!|"THE ""BEST"" OF ...|    28|
|       Jeopardy!|MOST POPULAR BABY...|    31|
|       Jeopardy!|         MYTHBUSTERS|    11|
|       Jeopardy!|          ADDRESS ME|    10|
|       Jeopardy!|  FILL IN THE ANIMAL|    18|
|       Jeopardy!|THE EMPIRE STRIKE...|    25|
|       Jeopardy!|"THE ""BEST"" OF ...|    28|
|       Jeopa

In [12]:
 from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
jeop_to_num = StringIndexer(inputCol='round',outputCol='label')
tokenizer = Tokenizer(inputCol="category", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [13]:
 from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [15]:

 # Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[jeop_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [16]:

# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [17]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[132740,2...|
|  1.0|(262145,[90843,92...|
|  1.0|(262145,[67117,12...|
|  1.0|(262145,[84025,19...|
|  1.0|(262145,[140197,2...|
|  1.0|(262145,[116782,2...|
|  1.0|(262145,[132740,2...|
|  1.0|(262145,[90843,92...|
|  0.0|(262145,[5078,262...|
|  0.0|(262145,[73199,22...|
|  0.0|(262145,[132804,2...|
|  0.0|(262145,[102177,1...|
|  0.0|(262145,[63956,98...|
|  0.0|(262145,[58625,26...|
|  0.0|(262145,[5078,262...|
|  0.0|(262145,[73199,22...|
|  0.0|(262145,[132804,2...|
|  0.0|(262145,[102177,1...|
|  0.0|(262145,[63956,98...|
|  0.0|(262145,[58625,26...|
+-----+--------------------+
only showing top 20 rows



In [18]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [19]:
 # Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)


+-----------+----------------+---------+----------+-----+----+-----+--------------------+---------------+------+-----+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|show_number|           round| air_date|  category|value|year|month|            question|         answer|length|label|  token_text| stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----------+----------------+---------+----------+-----+----+-----+--------------------+---------------+------+-----+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|         10|Double Jeopardy!|9/21/1984|      1789|  200|1984|    9|Washington procla...|   Thanksgiving|     4|  1.0|      [1789]|      [1789]|(262144,[35326],[...|(262144,[35326],[...|(262145,[35326,26...|[-164.3641

In [20]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.725462
